# Streaming historical fundamental data

## Used APIs

1.  `/api/v3/ratios/{ticker}`: This provides most of the profitability, per-share, and valuation ratios.
2.  `/api/v3/financial-growth/{ticker}`: This provides the year-over-year growth rates.

## Flow

1.  **Two API Calls:** The function makes separate `GET` requests to the `/ratios` and `/financial-growth` endpoints for the specified ticker.
2.  **Error Handling:** It uses `response.raise_for_status()` to catch network errors and also checks if the returned JSON contains an FMP-specific error message.
3.  **DataFrame Creation:** Each JSON response is converted into a pandas DataFrame, with the `date` column set as the index for easy alignment.
4.  **Joining Data:** The two DataFrames are joined together on their date index. An `inner` join ensures data integrity by only keeping rows for dates that are present in both results.
5.  **Column Selection and Renaming:** A dictionary `column_map` is used to select only the columns you asked for and to give them more readable names (e.g., `priceEarningsRatio` becomes `P/E Ratio`). This is a robust way to handle missing columns if the API doesn't return a specific ratio for a company.
6.  **Formatting Output:** In the main execution block, we use pandas' `Styler` object (`.style.format()`) to apply nice formatting, showing percentages for growth rates and margins, and two decimal places for other ratios.

In [34]:
import requests
import pandas as pd
import numpy as np
import time
import os
from datetime import datetime
import math
import yfinance as yf

def get_comprehensive_historical_ratios(api_key: str, ticker: str, period: str = "quarter", limit: int = 15) -> pd.DataFrame | None:
    # ... (code from your prompt) - Omitted for brevity
    print(f"\n--- Fetching Comprehensive Historical Data for {ticker} ---")
    base_url = 'https://financialmodelingprep.com/api/v3'
    params = {'apikey': api_key, 'period': period, 'limit': limit}
    try:
        ratios_endpoint = f'/ratios/{ticker}'
        ratios_response = requests.get(f"{base_url}{ratios_endpoint}", params=params)
        ratios_response.raise_for_status()
        ratios_data = ratios_response.json()
        if not ratios_data or 'Error Message' in ratios_data: return None
        ratios_df = pd.DataFrame(ratios_data).set_index('date')

        growth_endpoint = f'/financial-growth/{ticker}'
        growth_response = requests.get(f"{base_url}{growth_endpoint}", params=params)
        growth_response.raise_for_status()
        growth_data = growth_response.json()
        if not growth_data or 'Error Message' in growth_data: return None
        growth_df = pd.DataFrame(growth_data).set_index('date')

        combined_df = ratios_df.join(growth_df, lsuffix='_ratio', rsuffix='_growth', how='inner')

        column_map = {
            'returnOnEquity': 'ROE', 'returnOnAssets': 'ROA', 'grossProfitMargin': 'Gross Profit Margin',
            'eps': 'EPS', 'dividendYield': 'Dividend Yield', 'freeCashFlowPerShare': 'FCF per Share',
            'priceEarningsRatio': 'P/E Ratio', 'priceToBookRatio': 'P/B Ratio', 'priceToSalesRatio': 'P/S Ratio',
            'revenueGrowth': 'Revenue Growth', 'netIncomeGrowth': 'Net Income Growth',
            'freeCashFlowGrowth': 'FCF Growth', 'epsgrowth': 'EPS Growth'
        }
        final_columns = {api_name: new_name for api_name, new_name in column_map.items() if api_name in combined_df.columns}
        final_df = combined_df[list(final_columns.keys())].copy()
        final_df.rename(columns=final_columns, inplace=True)
        final_df.index = pd.to_datetime(final_df.index)
        final_df.sort_index(ascending=False, inplace=True)
        return final_df
    except Exception: return None

# Stream recent fundamental data

In [35]:
import requests
import pandas as pd
import time
import os
from datetime import datetime

def get_comprehensive_current_data(api_key: str, ticker: str) -> pd.DataFrame | None:
    """
    Fetches a snapshot of current fundamental and market data from multiple FMP
    endpoints and returns it as a single pandas DataFrame.

    Args:
        api_key (str): Your FMP API key.
        ticker (str): The stock ticker symbol.

    Returns:
        pd.DataFrame: A DataFrame containing the comprehensive current data,
                      with metrics as the index. Returns None if an error occurs.
    """
    print(f"\n--- Fetching Snapshot of Current Data for {ticker} ---")
    base_url = 'https://financialmodelingprep.com/api/v3'

    try:
        # --- 1. Fetch Data from 3 Separate Endpoints ---
        params = {'apikey': api_key}
        quote_resp = requests.get(f"{base_url}/quote/{ticker}", params=params)
        ttm_resp = requests.get(f"{base_url}/ratios-ttm/{ticker}", params=params)
        growth_params = {'apikey': api_key, 'period': 'annual', 'limit': 1}
        growth_resp = requests.get(f"{base_url}/financial-growth/{ticker}", params=growth_params)

        # Check for any request errors
        for resp in [quote_resp, ttm_resp, growth_resp]:
            resp.raise_for_status()

        quote_data = quote_resp.json()
        ttm_data = ttm_resp.json()
        growth_data = growth_resp.json()

        # --- 2. Validate and Extract Data ---
        if not all([quote_data, ttm_data, growth_data]):
            print(f"Incomplete data received for {ticker}.")
            return None

        quote = quote_data[0]
        ttm = ttm_data[0]
        growth = growth_data[0]

        # --- 3. Aggregate Data into a Dictionary ---
        data_dict = {
            # Market Data
            "Company Name": quote.get('name'),
            "Symbol": quote.get('symbol'),
            "Current Price": quote.get('price'),
            "Change": quote.get('change'),
            "Change %": quote.get('changesPercentage'),
            "Data Timestamp": datetime.fromtimestamp(quote.get('timestamp')).strftime('%Y-%m-%d %H:%M:%S'),

            # Valuation Ratios (TTM)
            "P/E Ratio": ttm.get('peRatioTTM'),
            "P/B Ratio": ttm.get('priceToBookRatioTTM'),
            "P/S Ratio": ttm.get('priceToSalesRatioTTM'),

            # Profitability & Per-Share (TTM)
            "Return on Equity (ROE)": ttm.get('returnOnEquityTTM'),
            "Return on Assets (ROA)": ttm.get('returnOnAssetsTTM'),
            "Gross Profit Margin": ttm.get('grossProfitMarginTTM'),
            "EPS (TTM)": ttm.get('epsTTM'),
            "Dividend Yield": ttm.get('dividendYieldTTM'),
            "FCF per Share": ttm.get('freeCashFlowPerShareTTM'),

            # Latest Reported Growth Rates (from last annual report)
            "Growth Report Date": growth.get('date'),
            "Revenue Growth": growth.get('revenueGrowth'),
            "Net Income Growth": growth.get('netIncomeGrowth'),
            "FCF Growth": growth.get('freeCashFlowGrowth'),
            "EPS Growth": growth.get('epsgrowth')
        }

        # --- 4. Convert to DataFrame and Return ---
        df = pd.DataFrame.from_dict(data_dict, orient='index', columns=['Value'])
        return df

    except requests.exceptions.RequestException as e:
        print(f"A network error occurred: {e}")
        return None
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return None

# Generate historical volatility

In [36]:
def calculate_historical_volatility(ticker: str, period: str, limit: int, window_size: int = 21) -> pd.DataFrame | None:
    # ... (code from your prompt) - Omitted for brevity
    print(f"\n--- Calculating Historical Volatility for {ticker} ---")
    if period.lower() == 'annual':
        history_duration = f"{limit}y"
    elif period.lower() == 'quarter' or period.lower() == 'quarterly': # Accepts both
        years_needed = math.ceil(limit / 4)
        history_duration = f"{years_needed}y"
    else: return None
    print(f"Input: {limit} '{period}' periods -> Fetching {history_duration} of price data.")
    try:
        stock_data = yf.Ticker(ticker).history(period=history_duration, auto_adjust=True)
        if stock_data.empty: return None
        stock_data.index = stock_data.index.tz_localize(None)
        stock_data['LogReturn'] = np.log(stock_data['Close'] / stock_data['Close'].shift(1))
        rolling_std = stock_data['LogReturn'].rolling(window=window_size).std()
        stock_data['AnnualizedVolatility'] = rolling_std * np.sqrt(252)
        result_df = stock_data[['Close', 'AnnualizedVolatility']].copy()
        result_df.dropna(inplace=True)
        return result_df
    except Exception as ex:
      print(ex)
      return None

In [45]:
def get_combined_historical_data(api_key: str, ticker: str, period: str = "quarter", limit: int = 20) -> pd.DataFrame | None:
    """
    Combines historical fundamental ratios with daily price and volatility data.

    This master function calls the helper functions to get low-frequency (ratios)
    and high-frequency (volatility) data, then merges them into a single,
    daily-indexed DataFrame using a forward-fill strategy.
    """
    print(f"\n{'='*20} STARTING COMBINED DATA FETCH FOR {ticker} {'='*20}")

    # 1. Get the low-frequency fundamental data (e.g., quarterly)
    ratios_df = get_comprehensive_historical_ratios(api_key, ticker, period, limit)
    if ratios_df is None:
        print("Failed to get historical ratios. Aborting combination.")
        return None

    # 2. Get the high-frequency daily price and volatility data
    volatility_df = calculate_historical_volatility(ticker, period, limit)
    if volatility_df is None:
        print("Failed to get historical volatility. Aborting combination.")
        return None

    print("\n--- Combining daily volatility with periodic fundamental data ---")

    # 3. Join the quarterly/annual ratios to the daily volatility data.
    # The daily data ('volatility_df') is the primary index.
    # This places fundamental data only on the exact report dates,
    # leaving all other days with 'NaN' in the fundamental columns. This is the desired behavior.
    combined_df = volatility_df.join(ratios_df)
    print(combined_df)

    # 4. Forward-fill the fundamental data. This is the key step.
    # It propagates the last known ratio value forward until a new one is
    # reported. This correctly fills the 'NaN' values from the previous step.
    combined_df.ffill(inplace=True)

    # 5. Drop any rows at the very beginning of the DataFrame that are still NaN.
    # This happens if the price history starts before the first financial report.
    combined_df.dropna(inplace=True)

    print(f"--- Combination complete. Final DataFrame has {len(combined_df)} rows. ---")
    print(f"{'='*20}   FINISHED COMBINED DATA FETCH FOR {ticker}   {'='*20}\n")

    return combined_df


In [44]:
# --- Main Execution ---
if __name__ == "__main__":
    # IMPORTANT: Replace with your FMP API Key
    FMP_API_KEY = "v0Y7rqjEfz0nBiixKBqJwLLgyFYbOUGA"
    TICKER_SYMBOL = "AAPL"
    DATA_PERIOD = "quarter"
    DATA_LIMIT = 12  # Fetch 12 quarters (3 years) of data

    master_df = get_combined_historical_data(
        api_key=FMP_API_KEY,
        ticker=TICKER_SYMBOL,
        period=DATA_PERIOD,
        limit=DATA_LIMIT
    )

    if master_df is not None:
        print("\n--- Master DataFrame Sample ---")

        # Display the last 5 rows to see the most recent data
        print("\nDataFrame Tail:")
        print(master_df)

        # To prove the forward-fill works, let's view data around a quarter-end date
        # Find the last report date in the data
        if not master_df.empty:
            last_date = master_df.index[-1]
            # Try to find a date like '2023-12-31' or '2024-03-31' near the end
            # This is a sample verification, the exact date might differ
            try:
                print("\n--- Verifying the forward-fill around a report date ---")
                print("Notice how ROE stays constant until the report date, then might change.")
                print(master_df.loc['2024-03-27':'2024-04-02'])
            except KeyError:
                print("\n(Could not find specific dates for verification, but tail shows data.)")


==================== STARTING COMBINED DATA FETCH FOR AAPL ====================

--- Fetching Comprehensive Historical Data for AAPL ---

--- Calculating Historical Volatility for AAPL ---
Input: 12 'quarter' periods -> Fetching 3y of price data.

--- Combining daily volatility with periodic fundamental data ---
                 Close  AnnualizedVolatility  ROE  ROA  Gross Profit Margin  \
Date                                                                          
2022-08-03  163.506836              0.268625  NaN  NaN                  NaN   
2022-08-04  163.191864              0.270522  NaN  NaN                  NaN   
2022-08-05  162.965210              0.264984  NaN  NaN                  NaN   
2022-08-08  162.492081              0.266716  NaN  NaN                  NaN   
2022-08-09  162.541351              0.257111  NaN  NaN                  NaN   

            Dividend Yield  FCF per Share  P/E Ratio  P/B Ratio  P/S Ratio  \
Date                                                 